#### Reg CAI
- look at the relationship between CAI and stability for all genes and for CTS TFs

In [ ]:
#Imports
import sys
import os
import pandas as pd
import seaborn as sns
import numpy as np
import pickle

sys.path.append('../scripts')
from plot_helpers import *
from annotation_utilities import *
from utilities import load_dataset
from plotting_fxns import PrettyBox

%load_ext autoreload
%autoreload 2

In [ ]:
outdir = '../Figures/Reg'
os.makedirs(outdir, exist_ok=True)

In [ ]:
#Load stability data
rate_df = load_dataset('../Figures/summary_files/INSPEcT_rates.csv', '../Figures/summary_files/brain4sU_passed.csv')

In [ ]:
CAI_file = '../../gene_attributes/CAI/CAI_values.csv'
cai_df = pd.read_csv(CAI_file, index_col='gene')

In [ ]:
rate_df['CAI'] = cai_df['CAI_hiexp']

In [ ]:
#Plot degradation rate vs. CAI
#I've clipped off a few of the really low degradation rate RNAs
#Should maybe set the lowest ones with halflife of 500 min to min deg rate to avoid issue
fig = plt.figure(figsize=(dfig,dfig), constrained_layout=True)
ax = fig.add_subplot(111)
ax = sns.histplot(x=rate_df['CAI'], y=rate_df['deg_rate'].apply(np.log10), cmap='rocket', ax=ax, zorder=2)
ax.set_ylabel('decay rate (log'r'$_{10}$'' 1 / min)')
ax.set_xlabel('codon adaptation index (CAI)')
ax.set_ylim(-5, 1)
plt.savefig('%s.%s' % (os.path.join(outdir, 'cai_deg'), out_fmt), dpi = out_dpi)# ax = sns.violinplot(data=me_df, x='CAI', y='category', order=order, orient='h', color=color_dict['grey'], ax = ax)

In [ ]:
me_df = pd.read_csv('../Figures/Devreg/gene_cat_me3.csv', index_col='gene')

In [ ]:
#Plot degradation rate vs. CAI with the updowngene TFs highlighted
from plotting_fxns import plot_grey_scatter
updowngene = me_df[me_df['category'] == 'updowngene'].index
rate_df['deg_log10'] = rate_df['deg_rate'].apply(np.log10)
#Need to dropna from the df, these correspond to non-coding RNAs
ax = plot_grey_scatter(x='CAI', y='deg_log10', data=rate_df.dropna(subset=['CAI']).copy(), genegroup=updowngene, grouplabel='updowngene')
ax.set_ylabel('decay rate (log'r'$_{10}$'' 1 / min)')
ax.set_xlabel('codon adaptation index (CAI)')
ax.set_ylim(-5, 1)
plt.savefig('%s.%s' % (os.path.join(outdir, 'cai_deg_tfs'), out_fmt), dpi = out_dpi)# ax = sns.violinplot(data=me_df, x='CAI', y='category', order=order, orient='h', color=color_dict['grey'], ax = ax)

In [ ]:
me_df['CAI'] = rate_df['CAI']
fig = plt.figure(figsize=(dfig*2,dfig), constrained_layout=True)
ax = fig.add_subplot(111)
# ax = sns.boxplot(x='category', y='CAI', data=me_df, ax=ax)
order = ['none','up', 'gene', 'down', 'upgene', 'downgene', 'updowngene']
ax = PrettyBox(data=me_df, x='CAI', y='category', order=order, orient='h', color=color_dict['grey'], fliersize=1, ax = ax)
pretty_names = {'up':'upstream (A)', 'down':'downstream (C)', 'gene':'gene body (B)', 'up':'upstream (A)', 'updown':'(AC)', 
               'upgene':'(AB)', 'downgene':'(BC)', 'updowngene':'(ABC)', 'none':'none'}

#prettify the category labels
new_labs = []
for i in ax.get_yticklabels():
    i.set_text(pretty_names[i.get_text()])
    new_labs.append(i)
ax.set_yticklabels(new_labs)

plt.savefig('%s.%s' % (os.path.join(outdir, 'cai_me3_box'), out_fmt), dpi = out_dpi)# ax = sns.violinplot(data=me_df, x='CAI', y='category', order=order, orient='h', color=color_dict['grey'], ax = ax)
#ax.set_ylim(0.5, 0.9)